In [10]:
import tensorflow as tf
from keras.applications import vgg16
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt

In [3]:
data_dir = "/Users/shirinshujaa/Furniture_Data"
desired_size = (256, 256)

image_data = []
image_hashes = set()
image_color_hist = []

main_folder_name = os.path.basename(data_dir)
# Define the selected categories
selected_categories = ['sofas', 'beds']

In [26]:
def preprocess_images(dataset_folder, desired_size, max_images_per_category):
    image_data = []

    for root, dirs, files in os.walk(dataset_folder):
        for parent_folder in dirs:
            parent_folder_path = os.path.join(root, parent_folder)
            
            category = os.path.basename(os.path.dirname(parent_folder_path))
            style = os.path.basename(parent_folder_path)

            category_images = []

            for filename in os.listdir(parent_folder_path):
                if filename == ".DS_Store":
                    continue

                file_path = os.path.join(parent_folder_path, filename)

                if os.path.isdir(file_path):
                    continue

                try:
                    img = cv2.imread(file_path)
                    if img is None:
                        raise Exception("Image is corrupted or invalid")
                    resized_img = cv2.resize(img, desired_size)
                    category_images.append((resized_img, category, style))  # Include category and style

                except Exception as e:
                    print(f"Error loading image {file_path}: {e}")

            # Sample a subset of images per category
            sampled_images = random.sample(category_images, min(max_images_per_category, len(category_images)))
            image_data.extend(sampled_images)

            # Display some sample images
            # if num_sample_images > 0 and sampled_images:
            #     print(f"Sample images from category: {category}, style: {style}")
            #     fig, axes = plt.subplots(1, min(len(sampled_images), num_sample_images), figsize=(12, 4))
            #     for i in range(min(len(sampled_images), num_sample_images)):
            #         title = f"Category: {category.capitalize()}, Style: {style.capitalize()}"
            #         axes[i].imshow(cv2.cvtColor(sampled_images[i][0], cv2.COLOR_BGR2RGB))
            #         axes[i].set_title(title)
            #         axes[i].axis('off')
            #     plt.show()

    return image_data


In [27]:
# Load and preprocess images
data_dir = "/Users/shirinshujaa/Furniture_Data"
desired_size = (256, 256)
max_images_per_category = 100  # Define the maximum number of images per category
num_sample_images=3
image_data = preprocess_images(data_dir, desired_size, max_images_per_category)

In [28]:
# Split data into features and labels
X = np.array([entry[0] for entry in image_data])
y = np.array([entry[1] for entry in image_data])

In [29]:
from keras.utils import to_categorical
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

In [30]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)


In [31]:
# Define VGG16 base model without the top (fully connected) layers
def create_vgg16_base(input_shape):
    vgg_base = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in vgg_base.layers:
        layer.trainable = False  # Freeze VGG16 base layers
    return vgg_base

In [32]:
# Create VGG16 base
input_shape = X_train.shape[1:]  # Input shape excluding batch dimension
vgg_base = create_vgg16_base(input_shape)


In [33]:
# Define the custom head for the model
def create_custom_head(bottom_model, num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024, activation='relu')(top_model)
    top_model = Dense(512, activation='relu')(top_model)
    top_model = Dense(num_classes, activation='softmax')(top_model)
    return top_model

In [34]:
# Create custom head
num_classes = y_categorical.shape[1]  # Number of classes in your dataset
custom_head = create_custom_head(vgg_base, num_classes)

In [35]:
# Combine VGG16 base and custom head to create the final model
model = Model(inputs=vgg_base.input, outputs=custom_head)

In [36]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [38]:
# Train the model
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
207/207 ━━━━━━━━━━━━━━━━━━━━ 711s 3s/step - accuracy: 0.7629 - loss: 1.6386 - val_accuracy: 0.9006 - val_loss: 0.2968
Epoch 2/5
207/207 ━━━━━━━━━━━━━━━━━━━━ 711s 3s/step - accuracy: 0.9391 - loss: 0.1778 - val_accuracy: 0.8994 - val_loss: 0.3291
Epoch 3/5
207/207 ━━━━━━━━━━━━━━━━━━━━ 715s 3s/step - accuracy: 0.9511 - loss: 0.1318 - val_accuracy: 0.8836 - val_loss: 0.3969
Epoch 4/5
207/207 ━━━━━━━━━━━━━━━━━━━━ 745s 4s/step - accuracy: 0.9623 - loss: 0.1059 - val_accuracy: 0.9188 - val_loss: 0.3042
Epoch 5/5
207/207 ━━━━━━━━━━━━━━━━━━━━ 751s 4s/step - accuracy: 0.9738 - loss: 0.0771 - val_accuracy: 0.9127 - val_loss: 0.4049


In [39]:
# Evaluate the model on the testing set
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

52/52 ━━━━━━━━━━━━━━━━━━━━ 151s 3s/step - accuracy: 0.9034 - loss: 0.4054
Test Loss: 0.40491169691085815
Test Accuracy: 0.9127272963523865


In [40]:
# Save the trained model
model.save("furniture_classification_model_Vgg16.h5")
print("Model saved successfully!")

Model saved successfully!
